In [1]:
import pandas as pd

PATH = '/home/kai/data/kaggle/homecredit/'

train = pd.read_csv(PATH + 'application_train.csv')
test = pd.read_csv(PATH + 'application_test.csv')
pos = pd.read_csv(PATH + 'POS_CASH_balance.csv')

'done'

'done'

In [2]:
pos.columns

Index(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'CNT_INSTALMENT',
       'CNT_INSTALMENT_FUTURE', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF'],
      dtype='object')

In [4]:
set_pos = set(pos.NAME_CONTRACT_STATUS)

In [5]:
for status in set_pos:
    print(status)
    print((pos['NAME_CONTRACT_STATUS'] == status).sum()/len(pos))

Returned to the store
0.0005460258496896122
Demand
0.0007064040703272496
Approved
0.0004916332362065232
Canceled
1.499796327658704e-06
Amortized debt
6.359136429272905e-05
Active
0.9149876446778528
Signed
0.0087248151700999
Completed
0.0744781858623599
XNA
1.999728436878272e-07


In [14]:
0.9149876446778528 + 0.0744781858623599

0.9894658305402126

In [11]:
_df = pos.sort_values(['SK_ID_CURR','SK_ID_PREV','MONTHS_BALANCE'], ascending = False)
_df.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
8514315,2729207,456255,-13,3.0,0.0,Completed,0,0
8606094,2729207,456255,-14,3.0,0.0,Active,0,0
573934,2729207,456255,-15,6.0,5.0,Active,0,0
8159633,2729207,456255,-16,6.0,6.0,Active,0,0
3069694,2631384,456255,-2,24.0,0.0,Completed,0,0


In [13]:
_df[100:

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
6824591,1686207,456253,-62,6.0,5.0,Active,0,0
8284797,1686207,456253,-63,6.0,6.0,Active,0,0
7961273,1668235,456253,-89,4.0,0.0,Completed,0,0
4358638,1668235,456253,-90,4.0,0.0,Active,0,0
2768848,1668235,456253,-91,4.0,1.0,Active,5,5
4224142,1668235,456253,-92,4.0,2.0,Active,0,0
4526384,1668235,456253,-93,4.0,3.0,Active,0,0
2414840,1668235,456253,-94,4.0,4.0,Active,0,0
1169973,1503599,456252,-76,6.0,0.0,Completed,0,0
4754066,1503599,456252,-77,6.0,1.0,Active,0,0


In [ ]:
1. Onehot encoding for status

2. SK_DPD and SK_DPD_DEF:
    count how many sk_dpd_def a prev id has
    sk_dpd:
        1. how many records, positive sk_dpd, sk_dpd_def a prev id has---> agg(records, skdpd, sk_dpd_def) of curr id
        2. pct of sk_dpd and sk_dpd in a prev id --> agg into curr
        3**. same for 1: check if prev id is completed in the end ---> num of completed, pct of completed into curr id
        4. find the (sum, mean, min, max, std) , agg, for all sk_dpd, sk_dpd_def; --> agg into curr (notice it is different from 1, as 1 is about # of dpd)
        5. growth rate: 
            1. for sk_dpd increasing, what is the mean for the increase
            2. for sk_dpd decreasing, count how many deceasing are there; what is the mean for the decrease
        6. unique count for count_installment --> agg(sum, mean, min, max, std) for curr id
        7. num of count for count_installment --> agg(sum, mean, min, max, std) for curr id
        8. last record for prev id: get future installment/count_installment --> curr id                                                 

3. Lastly do it with timeframe
    1. check for 12 months:
                                                                                                     
    2. check for 6 months:
        1. how many prev id, how many records for each prev id
        2. num of sk_dpd, sk_dpd_def
        3. sum of dpd and def





goal:
    more to interprete and to read
    less columns

In [31]:
pos = pos.sort_values(['SK_ID_CURR','SK_ID_PREV','MONTHS_BALANCE'], ascending = False)

x = pos.SK_DPD>5
pos_dpd = pos.loc[x]
y = pos.SK_DPD_DEF > 2
pos_def = pos.loc[y]

In [47]:

x = pos[pos['NAME_CONTRACT_STATUS'] == 'Completed']
set(x['CNT_INSTALMENT_FUTURE'])

{0.0, 7.0, 8.0, 10.0}

In [49]:
x[x['CNT_INSTALMENT_FUTURE'] ==8]
len(x)

744883

In [35]:
pos_dpd.head(200)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
4650657,1395578,456248,-83,6.0,1.0,Active,20,0
5933161,1395578,456248,-84,6.0,2.0,Active,19,0
2495422,1395578,456248,-85,6.0,3.0,Active,19,0
3951698,1395578,456248,-86,6.0,4.0,Active,18,0
1345238,2414642,456234,-29,6.0,2.0,Active,16,16
9967873,2583995,456200,-58,18.0,0.0,Active,11,11
5879917,2318679,456200,-67,12.0,8.0,Active,10,10
8356195,1978246,456187,-1,6.0,0.0,Active,3011,0
8356197,1978246,456187,-2,6.0,0.0,Active,2981,0
8356198,1978246,456187,-3,6.0,0.0,Active,2950,0


In [2]:
def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / (df[col2] + 1)

def positive_count(df, gp_col, col):
    group = (df[col] > 0).astype('int8')
    group = pd.concat([df[gp_col], group], axis=1).groupby(gp_col)[[col]].sum().reset_index().rename(index=str, columns={col: 'positivecount_'+'_'.join([gp_col, col])})
    return group.astype({gp_col: df[gp_col].dtype})

def count(df, gp_col, col):
    group = df[[gp_col,col]].groupby(gp_col)[[col]].count().reset_index().rename(index=str, columns={col:'count_'+gp_col})
    return group.astype({gp_col: df[gp_col].dtype})

def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index().astype({gp_col: df[gp_col].dtype})

def one_hot(df, gp_col, col):
    return pd.concat([df[gp_col], pd.get_dummies(df[col], prefix='onehot_' + col)], axis=1).groupby(gp_col).sum().reset_index().astype({gp_col: df[gp_col].dtype})

'done'

'done'

In [3]:
merged_col = 'SK_ID_CURR'
df = pd.concat([train['SK_ID_CURR'], test['SK_ID_CURR']], ignore_index=True).to_frame()
print(df.shape)
# minus
name1 = minus_name('SK_DPD', 'SK_DPD_DEF')
pos[name1] = minus(pos, 'SK_DPD', 'SK_DPD_DEF')

# ratio
numerator = ['SK_DPD', 'SK_DPD_DEF', name1]
denominator = ['MONTHS_BALANCE', 'CNT_INSTALMENT', 'CNT_INSTALMENT_FUTURE']
for x in numerator:
    for y in denominator:
#         print(x, y)
        pos[ratio_name(x, y)] = ratio(pos, x, y)

# one hot encoding sum
df = df.merge(one_hot(pos, merged_col, 'NAME_CONTRACT_STATUS'), on=merged_col, how='left')
print(df.shape)

# positive count
for x in ['SK_DPD', 'SK_DPD_DEF', name1]:
    df = df.merge(positive_count(pos, merged_col, x), on=merged_col, how='left')
print('pos', df.shape)
    
# count
df = df.merge(count(pos, merged_col, 'NAME_CONTRACT_STATUS'), on=merged_col, how='left')
count_name = [x for x in df.columns if 'count_' in x][0]
print(df.shape)

# categorical ratio
for x in df.columns:
    if 'positivecount_' in x or 'onehot_' in x:
#         print(x)
        df[ratio_name(x, count_name)] = ratio(df, x, count_name)
print(df.shape)
        
# numerical cols
ignored = ['SK_ID_CURR', 'SK_ID_PREV', 'NAME_CONTRACT_STATUS']
for x in set(pos.columns) - set(ignored):
#     print(x)
    df = df.merge(numerical(pos, merged_col, x, ['mean', 'median', 'sum', 'max', 'min', 'std']), on=merged_col, how='left')
print(df.shape)
    
columns = []
for x in df.columns:
    tmp = 'poscash_' + x if x != merged_col else x
    columns.append(tmp)
df.columns = columns
print(df.shape)

(356255, 1)
(356255, 10)
pos (356255, 13)
(356255, 14)
(356255, 26)
(356255, 116)
(356255, 116)


In [4]:
print(df.columns)
df.shape

Index(['SK_ID_CURR', 'poscash_onehot_NAME_CONTRACT_STATUS_Active',
       'poscash_onehot_NAME_CONTRACT_STATUS_Amortized debt',
       'poscash_onehot_NAME_CONTRACT_STATUS_Approved',
       'poscash_onehot_NAME_CONTRACT_STATUS_Canceled',
       'poscash_onehot_NAME_CONTRACT_STATUS_Completed',
       'poscash_onehot_NAME_CONTRACT_STATUS_Demand',
       'poscash_onehot_NAME_CONTRACT_STATUS_Returned to the store',
       'poscash_onehot_NAME_CONTRACT_STATUS_Signed',
       'poscash_onehot_NAME_CONTRACT_STATUS_XNA',
       ...
       'poscash_SK_DPD_minus_SK_DPD_DEF_divide_CNT_INSTALMENT_sum',
       'poscash_SK_DPD_minus_SK_DPD_DEF_divide_CNT_INSTALMENT_max',
       'poscash_SK_DPD_minus_SK_DPD_DEF_divide_CNT_INSTALMENT_min',
       'poscash_SK_DPD_minus_SK_DPD_DEF_divide_CNT_INSTALMENT_std',
       'poscash_SK_DPD_minus_SK_DPD_DEF_mean',
       'poscash_SK_DPD_minus_SK_DPD_DEF_median',
       'poscash_SK_DPD_minus_SK_DPD_DEF_sum',
       'poscash_SK_DPD_minus_SK_DPD_DEF_max',
       'pos

(356255, 116)

In [5]:
df.to_pickle(PATH + 'inter/poscash2curr.pkl')
'done'

'done'